In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X_train = pd.read_csv('../input/facial-keypoints-detection/training.zip')
X_test_original = pd.read_csv('../input/facial-keypoints-detection/test.zip')

In [ ]:
X_train.isna().sum()

In [ ]:
X_train = X_train.dropna()
y_train = X_train.drop(['Image'], axis=1)
X_train = X_train.Image

In [ ]:
print(X_train.head())
print('============================')
print(X_test_original.head())

In [ ]:
X_train = X_train.apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))

In [ ]:
X_test_original.columns

In [ ]:
X_test_original = X_test_original.Image

In [ ]:
X_test_original = X_test_original.apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape((96,96)))

In [ ]:
X_train /= 255.0
X_test_original /= 255.0

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image=plt.imshow(X_train[0])

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
y_train.loc[0]

In [ ]:
plt.imshow(X_train[0])
for i in range(1, 31, 2):
    plt.plot(y_train.loc[0][i-1], y_train.loc[0][i], 'ro')

In [ ]:
fig = plt.figure(figsize=(9, 9))

for i in range(9):
    ax = fig.add_subplot(3, 3, i + 1)
    plt.imshow(X_train[i])
    for i in range(1, 31, 2):
        plt.plot(y_train.loc[0][i-1], y_train.loc[0][i], 'ro')

plt.show()

In [ ]:
X_train = X_train.to_numpy()
X_test_original = X_test_original.to_numpy()

In [ ]:
X_train = X_train.flatten()
X_test_original = X_test_original.flatten()

In [ ]:
X_train = np.concatenate(X_train, axis=0)
X_train = np.concatenate(X_train, axis=0)

X_test_original = np.concatenate(X_test_original, axis=0)
X_test_original = np.concatenate(X_test_original, axis=0)

In [ ]:
X_train = X_train.reshape((-1, 96, 96))
X_test_original = X_test_original.reshape((-1, 96, 96))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.2, random_state=0)

In [ ]:
X_train = X_train.reshape((-1, 96, 96, 1))
X_valid = X_valid.reshape((-1, 96, 96, 1))
X_test = X_test.reshape((-1, 96, 96, 1))

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from tensorflow.keras import callbacks
from tensorflow import keras

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.1, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs'),
    early_stopping
]


In [ ]:
X_train = np.concatenate((X_train,X_train,X_train), axis=-1)
X_test = np.concatenate((X_test,X_test,X_test), axis=-1)
X_valid = np.concatenate((X_valid,X_valid,X_valid), axis=-1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
inputs = keras.Input(shape=(96, 96, 3))

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

res_model = ResNet50(input_tensor=inputs, weights='imagenet')

In [ ]:
len(res_model.layers)

In [ ]:
for layer in res_model.layers[:143]:
    layer.trainable = False

In [ ]:
model = keras.models.Sequential()
model.add(res_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(30, activation='relu'))

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.03)
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=150, callbacks=callbacks)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])

In [ ]:
loss, mae = model.evaluate(X_test, y_test)  # returns loss and metrics
print("loss: %.2f" % loss)
print("mae: %.2f" % mae)

In [ ]:
model.predict(X_test).shape

In [ ]:
X_test_original.shape

In [ ]:
X_test_original = X_test_original.reshape((-1,96,96,1))

In [ ]:
X_test_original = np.concatenate((X_test_original,X_test_original,X_test_original), axis=-1)

In [ ]:
X_test_original.shape

In [ ]:
predictions = model.predict(X_test_original)

In [ ]:
df = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
cols = y_train.columns

In [ ]:
cols = list(cols)

In [ ]:
df

In [ ]:
predictions.shape

In [ ]:
df.shape

In [ ]:
df = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')
for i in range(df.shape[0]):
     df.Location[i] = predictions[df.ImageId[i]-1][cols.index(df.FeatureName[i])]

In [ ]:
df.head()

In [ ]:
df.drop(['ImageId', 'FeatureName'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df = df.set_index(['RowId'])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# output = pd.DataFrame({'ImageId': indexes,
#                        'Label': predictions})
df.to_csv('submission.csv')

In [ ]:
df.Location.min()

In [ ]:
df.Location.max()

In [ ]:
def plot_test_image(df,prediction, index):
    image = plt.imshow(df[index])
    l = []
    for i in range(1,31,2):
        l.append(plt.plot(prediction[index][i-1], prediction[index][i], 'ro'))
        
    return image, l

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_test_image(X_test,predictions, i)

plt.show()